## Métodos útiles de criptografía para CTFs

In [5]:
c = 334757961089207699469910066584412431056902901407527274946225197457696082784065061767121578089638587991278450742947870751836796647007557910439331852822113624195403505379091163221959145542699755448010330474928229132513383028831737747387314906431593201511883892246073588416687789133895523134992186583468057991788596846849 # Criptograma
n = 445937235719970271756466297492751965991254917893534199198888997426639766569681232119818980875798790665128219008903036994361682569657213156218616443051707888786309084650643128814534960353080631853134961706746940693424872998067564101838686690650899568183639611110235238897618598536505276892483533487715888348406660509399 # Modulo
m = 0 # Mensaje
e = 65537 # Clave pública
d = 0 # Clave privada

In [1]:
def xgcd(a,b):
    """Extended GCD:
    Returns (gcd, x, y) where gcd is the greatest common divisor of a and b
    with the sign of b if b is nonzero, and with the sign of a if b is 0.
    The numbers x,y are such that gcd = ax+by."""
    prevx, x = 1, 0;  prevy, y = 0, 1
    while b:
        q, r = divmod(a,b)
        x, prevx = prevx - q*x, x  
        y, prevy = prevy - q*y, y
        a, b = b, r
    return a, prevx, prevy

In [2]:
import math
def modInv(n, a):
    """Calcula el inverso de a módulo n.
       Utiliza el algoritmo extendido de Euclides para ello.
 
    Args:
        a: número del que se calcula el módulo
        n: módulo del inverso
     
    Returns:
        inverso de a módulo n
 
    """
    mcd , u , v = xgcd(n,a)
    if mcd != 1:
        print("No existe inverso")
        return 0
     
    return mcd,u,v

In [4]:
# Install a pip package in the current Jupyter kernel
'''import sys
!{sys.executable} -m pip install pycryptodomex
!{sys.executable} -m pip install pycryptodome

######################################
#
# Signing and encrypting messages
#
######################################

# We import everything we will need
from Cryptodome.Signature import pss # Probabilistic Signature Scheme
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import RSA
from Cryptodome.Cipher import AES, PKCS1_OAEP # AES and Asymmetric cypher based on RSA and OAEP padding
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.Padding import pad, unpad


# Constants
RSA_KEY_LENGTH = 2048       # RSA key length, in bits
IV_LENGTH = 16              # Length of initialization vector, in bytes
SESSION_KEY_LENGTH = 32     # Session key length, in bytes
AES_BLOCK_SIZE = 16         # AES block size, in bytes
BITS_IN_BYTE = 8            # Number of bits in 1 byte
'''
import Crypto
from Crypto import *
from Crypto.Util import *

message = 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme...'

# Calculo de valores y claves para RSA 2048

p = Crypto.Util.number.getPrime(1024)
q = Crypto.Util.number.getPrime(1024)

N = p*q
phi = (p-1)*(q-1)

e = 65537
d = Crypto.Util.number.inverse(e, phi) # Inverso de e modulo phi

In [5]:
# Comprobamos que e y d con inversos modulo phi
e * d % phi

1

In [6]:
len(str(N))*4

2468

In [24]:
# Transforma el texto a hexadecimal y hexadecimal a entero, y eleva a la clave publica. Devuelve hexadecimal
def encrypt(mes, e):
    hexmes = mes.encode().hex()
    int_mes = int(hexmes, 16)
    encrypted = pow(int_mes, e, N) # int_mes ^ e mod N
    return hex(encrypted)

# Transforma hexadecimal a entero y eleva a la clave privada. Devuelve ASCII
def decrypt(intcipher, d):
    #intcipher = int(hexcipher, 16)
    decrypted_int = pow(intcipher, d, n)
    decrypted_hex = hex(decrypted_int)[2:]
    return bytes.fromhex(decrypted_hex).decode('ascii')

In [8]:
cipher = encrypt(message, e)
print("Texto cifrado: " + cipher)

Texto cifrado: 0x3a3c0ea8ffbcdcdac5b6ce6e26ae40c5e88676c871abe78446318eb47f44bb624810338ca0b671f2d655c3d634839ccb54e21b5d8f63918425ce86d6b9df4a260e9b641d12429e38d4ea3a725560c4111ebc3f541c497f626ec894ff848bfccf719b8ddda71481a4a8ba84ad9c34e791dbce4849253fdad5a31ab543d59305c5a3028b63cb71932d8551625f5021a42c58e298e6125f871edebf9d16103d5f1c4cf1bfac2ace6c96514bf6a34ed753c3103c1061966d8a6159bb2c37c8fe1be7048140474d34b85b836991eda7ac588eee76d601c2f9ba6d0c559d98c59eaf65f7ee1ac6b4f335937afe6feddddabe2905767a79a646f00807685936322d2ec6


In [9]:
dec = decrypt(cipher, d)
print("Descifrado: " + dec)

Descifrado: En un lugar de la Mancha, de cuyo nombre no quiero acordarme...


### Ataque de módulo común  
##### http://mikelgarcialarragan.blogspot.com/2017/01/criptografia-xli-ataque-rsa-mediante.html

In [36]:
n = 52841
e1= 11
e2= 7
c1= 7860
c2= 52061

In [37]:
_,s,t = xgcd(e1,e2); s,t

(2, -3)

In [38]:
_,_,c3 = modInv(n,c2)
c3

23101

In [39]:
a = pow(c1,s,n);a

8471

In [40]:
b = pow(c3,-t,n);b

17730

In [41]:
m = (a*b)%n;m

16708

In [43]:
decrypted_hex = hex(m)[2:]


In [44]:
bytes.fromhex(decrypted_hex).decode('ascii')

'AD'

In [3]:
key = 50483875293827348760936999826491042665018907273940663423292829786877694846533471940285381951256868974863063210369697584228853056515703611630158352391758927681031497808523072178015920094310803127882934133164610358872043611067235470062283155913351271728452986911272960600583120746857929751356068128698299832181681486268577786218071715875128341412983542682430122980249959493069837879179638711717117218367724736453338101341359926699392154650088234399922368935715093329509395116828570206637514703467232812490053414739132399987777586979283521501784752805329975339034199985515428242213854319829096734620501234
p = 91514892819797033705179399571227611692092856913300377973266729140373382315305352325947520546767428375542736209947536010920931094235256987366602022333668598445943676897129270112929569042756146444855210872641963540227487080469555040640668407120910156044105141128559028329041777867127037483908930283653500859907442430934763996059464170608731760771244982499063459386776776115639036656414141326882715090187673746563880543318956483611125878045563045085767103391255891846764748289144269428758133873311885447854229528568813923719146212759974019867134361918383584469265829718289574757937412545468378304435902459

inverse = modInv(p, key)

print(inverse)

(1, 19029447952762590536498191890684727222085462934078534780896248361883760523653855867538621070796436374467120955767632078033598923129326342337912842515435074995298051819759378750954514024045658411846477847111475725348885442727480601385865387617940201100474094760081587762959222037084065131583460414807560369818705629860173122994597866717073388587422305503797436628553574986336301952407570128925806749659376336370083870305143729927965864394694871897805322053887303917383426525470982989492344829106140869098579665904707544393545948135346880784111459964152257942169853912466998228109221552571478933341984229, -3449572521287613275376586000831882369325453331059354116229389340309851028645675732649462642449112025007287416160918337864057306258158356901787085516663348953382947761260688441217249183641234268242942126219026112957156372798275843001276497899530606394635140842509359987314262937786118741790032253770036701393576780894981619076117224457967800237023880282604132457011010684480717402716548276870

### Factorizar n y resolver 
##### https://www.alpertron.com.ar/ECM.HTM

In [17]:
n = 26776423251058594618720430940111309399082560058457455400757276683944987683699639419039205798616312527913498450416663902739864211292444707925281787807460603460877235590941855166979447583797852871053644187690198378033131944647242763665931238517292896653092604754797071032753695708558866133866267720619136063659148877126571641981697020871426038204799119070239300502174654327183618897039826908335349242017933196575486592484786149082336124819220387056151133281258875554820642003178322217882186580571313268386102841871794987802361635287257923789173422217914145877381633828687843949660325734143183476344433200091249125647393

In [1]:
p,q = 163635030635431466645458744796002909908647215873613947295996181900869360800400033263581571272503065118904651463297952384120999697060028804272898418586691879607611657735837748266534692036505364040855251985110317345406628583217211803323176146608538701077202651651334269565950400367181957150372801864201895294389,163635030635431466645458744796002909908647215873613947295996181900869360800400033263581571272503065118904651463297952384120999697060028804272898418586691879607611657735837748266534692036505364040855251985110317345406628583217211803323176146608538701077202651651334269565950400367181957150372801864201895295037
p,q

(163635030635431466645458744796002909908647215873613947295996181900869360800400033263581571272503065118904651463297952384120999697060028804272898418586691879607611657735837748266534692036505364040855251985110317345406628583217211803323176146608538701077202651651334269565950400367181957150372801864201895294389,
 163635030635431466645458744796002909908647215873613947295996181900869360800400033263581571272503065118904651463297952384120999697060028804272898418586691879607611657735837748266534692036505364040855251985110317345406628583217211803323176146608538701077202651651334269565950400367181957150372801864201895295037)

In [2]:
phi = (p-1)*(q-1); phi

26776423251058594618720430940111309399082560058457455400757276683944987683699639419039205798616312527913498450416663902739864211292444707925281787807460603460877235590941855166979447583797852871053644187690198378033131944647242763665931238517292896653092604754797071032753695708558866133866267720619136063658821607065300779048406103381834032384981824638492072607582661963381880175439026841808186099472927066337677289558190244314094125425100329447605336444085491795605418687706646721349117196498302540304392337901574353111548378120823500182527069924697068475227228525385175410528424933408819562043687596362845335057968

In [7]:
e = 65537

In [8]:
_,_,d = modInv(phi,e)

In [9]:
d

9484942334457257336673860632538627762938517658072383953012499476902862338481882434548349216700149462677752514250314364436973734915453925179446979629067517728066054659867176826852432605365933051581707277068342392023424296122583285144339742468818447530426245012475609259279140727744542430958167220565073831848261952912415240026378193844839969205446588323887170080794535872559170365943162002114485562342859786762121828541028526202781560366567040727011579497832441094877394370128474047272834516635001502558348232668340763347187017990980935299714145113475478655605232314826996157215273583305396129405438264637738292161233

In [10]:
(d*e)%phi

1

In [32]:
import Crypto
from Crypto import *
from Crypto.Util import *

ct = 716655107847297821001077235738210272906382106725640971027199454810171381017698735373821057297731121087138867910971991224310170102614090704161104897256685710072565163907235571031107353577697738712311510171994557105716680408971665589877387979710273871151121037297481061107137122951037388691261097257481031067387351211117388695910571100688210773531061151127235106118106738797829971999560907110086971047256631061097166334898723363701027138101851097042336710072355785106729063701017010210810497701029910187719912260987353821031037233637910272901197310373539788977256519510171385267108723582881027235731001057010380638972359711510771667112297701038151102735312312611170428060987042801168472901015710373539711210773873511211072908512490738879531097297731091047290100958871668011685701021088986738735121113704210557101725651438872359711210271999654103701035511398729757112105729063761037110086100106716610443100729010151103725748100107704211454105738797761037387123381097387978298735311538114735382100107738797761037297731009971668040897138110821067235731031057166803610071999670106713811082102716696518973535959108729011070102713743438670411259883723557851067010390571027166431248871386873104719911343102729757821047010380368873887859103716680409870413786867010212511910070102379886723557821037235579710572977310610173531061211077199455110671374363887233637310471379563897042335499704112510187735357881047388693510573531061031027042335110472331189587716655122907353485187704161110897290407098729010012486738759103102701021251169970103711198670428040987297571121097110077549973537310310271661046090711008673102704280409972351153810971664312497704210412498729085439070423354101711008676107738853123116711007788110729040761047235737910073534851887010261104857153

In [30]:
c = Crypto.Util.number.bytes_to_long(ct.encode('ascii'));
c

182451880315860160829220952489421062789866233804694077388996067709127314538663223040908460852817162659399599316220520726810140206622803251491531117806605001532738863293203108668253057725481043568358787846089941867997059862437393977230755454069845508205988887921070976496607944930694770003864060137510750910127514006595923662988566619081748215631536673460287257125946989880850692438512672410478499369196892043957284900942850331341656374568637963338020181763751336429733091207590736329548705874551667344837126602689586001055757158155601330717618825882912857687577546509555548879516441127143261035973363092462940947462128781017092312324032011444899903

In [33]:
decrypt(ct,d)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xd3 in position 1: ordinal not in range(128)